# Bitcoin & Twitter

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql import *
from pyspark.sql.functions import col
from pyspark.sql import functions as F


sc = SparkContext("local").getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

21/10/23 00:35:48 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp59s0)
21/10/23 00:35:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/23 00:35:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
#BITCOIN

# Unix Timestamp - Date represented as epoc value
# Date - date and time when the data point was collected
# Symbol - Symbol of the currency
# Open - Open value of the currency
# High - Highest value of currency in the given minute
# Low - Lowest value of currency in the given minute
# Close - Close value of the currency in the given minute
# Volume - Volume of the currency transacted in the given minute.

# TWITTER

# user_name	The name of the user, as they’ve defined it.
# user_location	The user-defined location for this account’s profile.
# user_description	The user-defined UTF-8 string describing their account.
# user_created	Time and date, when the account was created.
# user_followers	The number of followers an account currently has.
# user_friends	The number of friends a account currently has.
# user_favourites	The number of favorites a account currently has
# user_verified	When true, indicates that the user has a verified account
# date	UTC time and date when the Tweet was created
# text	The actual UTF-8 text of the Tweet
# hashtags	All the other hashtags posted in the tweet along with #Bitcoin & #btc
# source	Utility used to post the Tweet, Tweets from the Twitter website have a source value - web
# is_retweet	Indicates whether this Tweet has been Retweeted by the authenticating user.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
import seaborn as sns
from scipy import stats as stat

In [4]:
tweets_schema = StructType([
StructField('user_name', StringType(), True),
StructField('user_location', StringType(), True),
StructField('user_description', StringType(), True),
StructField('user_created', StringType(), True),
StructField('user_followers', StringType(), True),
StructField('user_friends', StringType(), True),
StructField('user_favourites', StringType(), True),
StructField('user_verified', StringType(), True),
StructField('date', StringType(), True),
StructField('text', StringType(), True),
StructField('hashtags', StringType(), True),
StructField('source', StringType(), True),
StructField('is_retweet', StringType(), True),
])

tweets_df = spark.read.csv(
    "./data/tweets.csv", 
    header=True, 
    schema=tweets_schema)

btc_schema = StructType([
StructField('Unix Timestamp', StringType(), True),
StructField('Date', StringType(), True),
StructField('Symbol', StringType(), True),
StructField('Open', FloatType(), True),
StructField('High', FloatType(), True),
StructField('Low', FloatType(), True),
StructField('Close', FloatType(), True),
StructField('Volume', FloatType(), True),
])

btc_df = spark.read.csv(
    "./data/bitcoinvalue.csv",
    sep=',',
    # header=True, 
    schema=btc_schema)

tweets_df.show(10)

+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|           user_name|  user_location|    user_description|       user_created|user_followers|user_friends|user_favourites|user_verified|               date|                text|            hashtags|              source|         is_retweet|
+--------------------+---------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+--------------------+-------------------+
|       DeSota Wilson|    Atlanta, GA|Biz Consultant, r...|2009-04-26 20:05:09|        8534.0|        7605|           4838|        False|2021-02-10 23:59:04|Blue Ridge Bank s...|         ['bitcoin']|     Twitter Web App|              False|
|            CryptoND|           nul

In [ ]:
#Function BEFORE

tweets_df["hashtags"]=tweets_df["hashtags"].apply(lambda s : s[1:-1].split(',') if isinstance(s,str) else [])
tweets_df["hashtags_len"]=tweets_df["hashtags"].apply(len)

#Function AFTER

tweets_df = tweets_df.select(F.split(col('hashtags'), ",").alias("hastags_array")).drop('hashtags')

In [ ]:
tweets_vol = tweets_df.groupby('date').size().rolling(10).mean().dropna()
fig, ax = plt.subplots()

ax.plot(tweets_vol.index,tweets_vol,label='Total Tweets')
ax.set_yscale('log')
ax.tick_params(axis='y')

lines, labels = ax.get_legend_handles_labels()

ax.legend(lines, labels, loc=0)
plt.title("Total Tweets / Date")

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(btc_value_df['Date'],btc_value_df['Open'],label='Bitcoin Value')
ax.tick_params(axis='y')

lines, labels = ax.get_legend_handles_labels()

ax.legend(lines, labels)
plt.title("Bitcoin Value")

plt.show()

In [ ]:
df_location = tweets_df.groupBy('user_location').count().sort(col("count").desc()).show()